# Titanic Survival Prediction

## Imports

In [98]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
)

## Exploration

In [99]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [100]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [101]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Feature Engineering

In [102]:
combined = pd.concat([train_df, test_df], sort=False)

In [103]:
combined['FamilySize'] = combined['SibSp'] + combined['Parch'] + 1
combined['IsAlone'] = (combined['FamilySize'] == 1).astype(int)

combined['Sex'] = combined['Sex'].map({'male': 0, 'female': 1})
combined['Embarked'] = combined['Embarked'].fillna('S').map({'S': 0, 'C': 1, 'Q': 2})
combined['Age'] = combined['Age'].fillna(combined['Age'].median())
combined['Fare'] = combined['Fare'].fillna(combined['Fare'].median())

combined['Title'] = combined['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
combined['Title'] = combined['Title'].replace(['Lady', 'Countess','Capt','Col','Don','Dr','Major',
                                               'Rev','Sir','Jonkheer','Dona'], 'Rare')
combined['Title'] = combined['Title'].map({"Mr":1, "Miss":2, "Mrs":3, "Master":4, "Rare":5})
combined['Title'] = combined['Title'].fillna(0)

combined.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

train_df = combined[:len(train_df)]
test_df = combined[len(train_df):]

X = train_df.drop(['Survived', 'PassengerId'], axis=1)
y = train_df['Survived']
X_test_final = test_df.drop(['Survived', 'PassengerId'], axis=1)

## Train Test Split

In [104]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [120]:
random_forest_model = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=10)

random_forest_model.fit(X_train, y_train)

y_pred = random_forest_model.predict(X_val)

print("Validation Accuracy (RandomForest):", accuracy_score(y_val, y_pred))
print("\nValidation Metrics (RandomForest):")
print(confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred))

Validation Accuracy (RandomForest): 0.8603351955307262

Validation Metrics (RandomForest):
[[94 11]
 [14 60]]
              precision    recall  f1-score   support

         0.0       0.87      0.90      0.88       105
         1.0       0.85      0.81      0.83        74

    accuracy                           0.86       179
   macro avg       0.86      0.85      0.86       179
weighted avg       0.86      0.86      0.86       179



In [121]:
random_forest_model.fit(X, y)
y_test_pred = random_forest_model.predict(X_test_final)

In [122]:
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': y_test_pred.astype(int)
})

submission.to_csv('submission.csv', index=False)
print("Submission file saved as: submission.csv")

print("Survival counts in submission:")
print(submission['Survived'].value_counts())

Submission file saved as: submission.csv
Survival counts in submission:
Survived
0    271
1    147
Name: count, dtype: int64
